In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2021-09-22_raw.csv to 2021-09-22_raw.csv


In [ ]:
uploaded_sec = files.upload()

Saving merge_clean_v2.csv to merge_clean_v2.csv


In [29]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx
import numpy as np
import ast
import nltk
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud

In [30]:
total_filename = '/content/merge_clean_v5.csv'
orig_filename = '/content/raw.csv'

In [31]:
df = pd.read_csv(total_filename, encoding="utf-8")
df = df.set_index('orig_index')
df = df.sort_values(by=['orig_index'])
orig_df = pd.read_csv(orig_filename, encoding="utf-8")
orig_df['verdict'] = 'INVALID'

In [32]:
orig_indexes = df.index
for i in orig_indexes:
  tweet = df.loc[i].tweet
  indexes = orig_df.loc[orig_df.tweet == tweet].index
  for j in indexes:
    orig_df.at[j,'verdict'] = df.loc[i].verdict


df = orig_df
df

,author id,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,verdict
0,630857697.0,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"RT @pcoogov: TINGNAN: Kahapon, Setyembre 20, i...",TRUE
1,630857697.0,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for Android,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,TRUE
2,630857697.0,PIACaraga,2021-09-21 23:53:17+00:00,,1.440000e+18,tl,0.0,0.0,0.0,3.0,Twitter for Android,"RT @pcoogov: TINGNAN: Nitong Lunes, Setyembre ...",INVALID
3,630857697.0,PIACaraga,2021-09-21 23:52:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,3.0,Twitter for Android,RT @pcoogov: TINGNAN: Ipinabatid ni Presidenti...,INVALID
4,630857697.0,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"RT @pcoogov: PANOORIN: Kahapon, Setyembre 20, ...",TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9232,1111.0,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Department of Health (DOH) said that child...,FALSE
9233,1111.0,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filipino official branded poor people a ‘burde...,FALSE
9234,1111.0,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presidential Spokesperson Harry Roque said the...,FALSE
9235,1111.0,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical trials in Indonesia recorded Sinovac’...,FALSE


In [33]:
df.verdict.value_counts()

INVALID         4017
TRUE            3099
MOSTLY_TRUE     1309
INVALIDT         492
MOSTLY_FALSE     170
FALSE            150
Name: verdict, dtype: int64

In [34]:
df = df[(df.verdict == 'TRUE') | (df.verdict == 'FALSE') | (df.verdict == 'MOSTLY_TRUE') | (df.verdict == 'MOSTLY_FALSE') | (df.verdict == 'HALF_TRUE')]
df = df.reset_index()
df = df.drop(columns=['index'])
#df['verdict'] = df['verdict'].astype('bool')
df

,author id,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,verdict
0,6.308577e+08,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"RT @pcoogov: TINGNAN: Kahapon, Setyembre 20, i...",TRUE
1,6.308577e+08,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for Android,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,TRUE
2,6.308577e+08,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"RT @pcoogov: PANOORIN: Kahapon, Setyembre 20, ...",TRUE
3,1.280000e+18,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for iPhone,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,TRUE
4,1.340000e+18,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Twitter for Android,RT @WinGatchalian74: Dapat mayroong sapat na a...,MOSTLY_TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4723,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Department of Health (DOH) said that child...,FALSE
4724,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filipino official branded poor people a ‘burde...,FALSE
4725,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presidential Spokesperson Harry Roque said the...,FALSE
4726,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical trials in Indonesia recorded Sinovac’...,FALSE


In [35]:
df['RT'] = False
df['RTUser'] = 'NA'
df['source_user'] = 'NA'
df['target_user'] = 'NA'
df['mentions'] = 'NA'

###Segregating People Who Retweeted, and the Usernames. Adding into source_user and target_user as well

In [36]:
for i in df.index:
  i = int(i)
  row = df.iloc[i]
  username_df = row.tweet.split(" ")
  if username_df[0] == 'RT':
    username_rt = str(username_df[1])[1:-1]
    df.at[i,'RT'] = True
    df.at[i,'RTUser'] = username_rt
    df.at[i,'source_user'] = username_rt
    df.at[i,'target_user'] = row.username
    
df

,author id,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,verdict,RT,RTUser,source_user,target_user,mentions
0,6.308577e+08,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"RT @pcoogov: TINGNAN: Kahapon, Setyembre 20, i...",TRUE,True,pcoogov,pcoogov,PIACaraga,NA
1,6.308577e+08,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for Android,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,TRUE,True,pcoogov,pcoogov,PIACaraga,NA
2,6.308577e+08,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"RT @pcoogov: PANOORIN: Kahapon, Setyembre 20, ...",TRUE,True,pcoogov,pcoogov,PIACaraga,NA
3,1.280000e+18,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for iPhone,RT @pcoogov: TINGNAN: Nakatanggap ng additiona...,TRUE,True,pcoogov,pcoogov,YamauchiElena,NA
4,1.340000e+18,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Twitter for Android,RT @WinGatchalian74: Dapat mayroong sapat na a...,MOSTLY_TRUE,True,WinGatchalian74,WinGatchalian74,greg_kassie,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4723,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Department of Health (DOH) said that child...,FALSE,False,NA,NA,NA,NA
4724,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filipino official branded poor people a ‘burde...,FALSE,False,NA,NA,NA,NA
4725,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presidential Spokesperson Harry Roque said the...,FALSE,False,NA,NA,NA,NA
4726,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical trials in Indonesia recorded Sinovac’...,FALSE,False,NA,NA,NA,NA


In [37]:
def remove_rt(tweet_row):
  tweet = ''
  split_str = tweet_row.split(" ")
  if split_str[0]=='RT':
    split_str.pop(0)
    split_str.pop(0)
    tweet = ' '.join([word for word in split_str])
    return tweet
  else:
    return tweet_row

In [38]:
df['tweet'] = df['tweet'].apply(lambda x: remove_rt(x))
df

,author id,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,verdict,RT,RTUser,source_user,target_user,mentions
0,6.308577e+08,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,pcoogov,PIACaraga,NA
1,6.308577e+08,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for Android,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,pcoogov,PIACaraga,NA
2,6.308577e+08,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,pcoogov,PIACaraga,NA
3,1.280000e+18,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for iPhone,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,pcoogov,YamauchiElena,NA
4,1.340000e+18,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Twitter for Android,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,WinGatchalian74,greg_kassie,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4723,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Department of Health (DOH) said that child...,FALSE,False,NA,NA,NA,NA
4724,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filipino official branded poor people a ‘burde...,FALSE,False,NA,NA,NA,NA
4725,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presidential Spokesperson Harry Roque said the...,FALSE,False,NA,NA,NA,NA
4726,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical trials in Indonesia recorded Sinovac’...,FALSE,False,NA,NA,NA,NA


### Segregating Mentions and placing the remaining source_user and target_user

In [39]:
for i in df.index:
  row = df.iloc[i]
  if row.RT == False:
    username_s = row.username
    mentions = [word[1:] for word in row.tweet.split(" ") if "@" in word]
    df.at[i,'source_user'] = username_s
    if mentions != []:
      df.at[i,'mentions'] = mentions
      df.at[i,'target_user'] = mentions
    

df

,author id,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,verdict,RT,RTUser,source_user,target_user,mentions
0,6.308577e+08,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,pcoogov,PIACaraga,NA
1,6.308577e+08,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for Android,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,pcoogov,PIACaraga,NA
2,6.308577e+08,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,Twitter for Android,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,pcoogov,PIACaraga,NA
3,1.280000e+18,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,Twitter for iPhone,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,pcoogov,YamauchiElena,NA
4,1.340000e+18,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Twitter for Android,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,WinGatchalian74,greg_kassie,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4723,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Department of Health (DOH) said that child...,FALSE,False,NA,poynter,NA,NA
4724,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filipino official branded poor people a ‘burde...,FALSE,False,NA,poynter,NA,NA
4725,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presidential Spokesperson Harry Roque said the...,FALSE,False,NA,poynter,NA,NA
4726,1.111000e+03,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical trials in Indonesia recorded Sinovac’...,FALSE,False,NA,poynter,NA,NA


###Dropping the columns not needed for the model

In [40]:
df = df.drop(columns=['author id', 'RTUser', 'mentions', 'source' ])
df

,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,tweet,verdict,RT,source_user,target_user
0,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,PIACaraga
1,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,PIACaraga
2,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,PIACaraga
3,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,YamauchiElena
4,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,greg_kassie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4723,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Department of Health (DOH) said that child...,FALSE,False,poynter,NA
4724,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filipino official branded poor people a ‘burde...,FALSE,False,poynter,NA
4725,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Presidential Spokesperson Harry Roque said the...,FALSE,False,poynter,NA
4726,poynter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clinical trials in Indonesia recorded Sinovac’...,FALSE,False,poynter,NA


### Duplicating rows with multiple target_users

In [41]:
x = len(df.index)

for i in df.index:
  row = df.iloc[i]
  if type(row.target_user) is list:
    for j in range(1,len(row.target_user)):
      df = df.append(row,ignore_index=True)
      if "@" in row.target_user[j]:
        df.at[x,'target_user'] = df.at[x,'target_user'] = row.target_user[j][1:]
      else:
        df.at[x,'target_user'] = row.target_user[j]
      x += 1
    df.at[i,'target_user'] = row.target_user[0]


df

,username,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,tweet,verdict,RT,source_user,target_user
0,PIACaraga,2021-09-21 23:53:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"TINGNAN: Kahapon, Setyembre 20, inilabas ni @n...",TRUE,True,pcoogov,PIACaraga
1,PIACaraga,2021-09-21 23:53:27+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,PIACaraga
2,PIACaraga,2021-09-21 23:52:23+00:00,,1.440000e+18,tl,0.0,0.0,0.0,2.0,"PANOORIN: Kahapon, Setyembre 20, nagbigay ng u...",TRUE,True,pcoogov,PIACaraga
3,YamauchiElena,2021-09-21 20:45:11+00:00,,1.440000e+18,tl,0.0,0.0,0.0,6.0,TINGNAN: Nakatanggap ng additional doses ng Pf...,TRUE,True,pcoogov,YamauchiElena
4,greg_kassie,2021-09-21 16:28:08+00:00,,1.440000e+18,tl,0.0,0.0,0.0,34.0,Dapat mayroong sapat na abiso sa mga lugar na ...,MOSTLY_TRUE,True,WinGatchalian74,greg_kassie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4862,roysant51162134,2021-09-17 08:29:20+00:00,,1.440000e+18,tl,0.0,0.0,1.0,0.0,@EdsonCGuido @ABSCBNNews habang dumadami nabab...,MOSTLY_FALSE,False,roysant51162134,ABSCBNNews
4863,dzrhnews,2021-09-17 04:23:36+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,Pakiusap na payagan na rin ang mga naka-unang ...,TRUE,False,dzrhnews,liezel_once
4864,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,inquirerdotnet
4865,GrowlHarhar,2021-09-17 03:09:24+00:00,,1.440000e+18,tl,0.0,0.0,0.0,0.0,@CityMouse14 @inquirerdotnet @KAguilarINQ Ikaw...,MOSTLY_TRUE,False,GrowlHarhar,KAguilarINQ


In [42]:
df.to_csv('from_model_v2.csv')